In [11]:
# Cell 1: Imports and Data Loading
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sns.set(style="whitegrid")

# Load Brent oil price data, filter 2015–2022, and downsample to monthly averages
df = pd.read_csv("../data/BrentOilPrices.csv")
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True, errors='coerce')
df = df[df['Date'] >= '2015-01-01']
df = df.set_index('Date').resample('ME').mean().reset_index()
prices = df['Price'].values
dates = df['Date'].values

# Load events data
events = pd.read_csv("../data/events.csv")
events['Date'] = pd.to_datetime(events['Date'], format='%d-%b-%y')

# Cell 2: Bayesian Change Point Model
with pm.Model() as model:
    # Define priors
    tau = pm.DiscreteUniform("tau", lower=0, upper=len(prices)-1)  # Change point index
    mu_1 = pm.Normal("mu_1", mu=50, sigma=20)  # Mean before change
    mu_2 = pm.Normal("mu_2", mu=50, sigma=20)  # Mean after change
    sigma = pm.HalfNormal("sigma", sigma=10)    # Common standard deviation

    # Switch function for mean
    mu = pm.math.switch(tau >= np.arange(len(prices)), mu_1, mu_2)

    # Likelihood
    observation = pm.Normal("obs", mu=mu, sigma=sigma, observed=prices)

    # Sampling
    trace = pm.sample(200, tune=100, return_inferencedata=False)

# Cell 3: Analyze Results
# Plot posterior distribution of tau
plt.figure(figsize=(10, 6))
plt.hist(trace['tau'], bins=20, density=True, alpha=0.7)
plt.title("Posterior Distribution of Change Point (2015–2022, Monthly)")
plt.xlabel("Time Index")
plt.ylabel("Density")
plt.savefig("../plots/tau_posterior.png")
plt.close()

# Convert tau to date
most_likely_tau = int(np.median(trace['tau']))
change_point_date = pd.Timestamp(dates[most_likely_tau])  # Convert to Timestamp
mu_1_mean = trace['mu_1'].mean()
mu_2_mean = trace['mu_2'].mean()
price_change = ((mu_2_mean - mu_1_mean) / mu_1_mean) * 100

# Find closest event
events['Date_Diff'] = abs(events['Date'] - change_point_date)
closest_event = events.loc[events['Date_Diff'].idxmin()]

# Cell 4: Print Results
results = f"""
Change Point Analysis Results (2015–2022, Monthly):
- Most Likely Change Point: {change_point_date.strftime('%d-%b-%Y')} (Index: {most_likely_tau})
- Mean Price Before: ${mu_1_mean:.2f}
- Mean Price After: ${mu_2_mean:.2f}
- Price Change: {price_change:.2f}%
- Closest Event: {closest_event['Event_Description']} on {closest_event['Date'].strftime('%d-%b-%Y')}
"""
print(results)

# Save results
with open("../task2_results.txt", "w") as f:
    f.write(results)

# Cell 5: Plot Price Series with Change Point
plt.figure(figsize=(12, 6))
plt.plot(dates, prices, label="Brent Oil Price (Monthly Avg)")
plt.axvline(change_point_date, color='red', linestyle='--', label=f"Change Point ({change_point_date.strftime('%d-%b-%Y')})")
plt.title("Brent Oil Prices with Detected Change Point (2015–2022, Monthly)")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.savefig("../plots/change_point.png")
plt.close()

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu_1, mu_2, sigma]


Output()

Sampling 2 chains for 100 tune and 200 draw iterations (200 + 400 draws total) took 344 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details



Change Point Analysis Results (2015–2022, Monthly):
- Most Likely Change Point: 30-Sep-2017 (Index: 32)
- Mean Price Before: $49.88
- Mean Price After: $69.21
- Price Change: 38.75%
- Closest Event: OPEC agrees to production cuts on 30-Nov-2016

